# सुत्र मॉडल का उपयोग हिंदी भाषा के लिए OpenAI क्लाइंट के साथ

<img src="https://play-lh.googleusercontent.com/_O9p4Z4yucA2NLmZBu9mTJCuBwXeT9NcbtrDN6I8gKlkIPRySV0adOmbyipjSj9Gew" width="150">

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/12clKIF1dkeUXkkTdDG9qlCM6tnmfLpPp?usp=sharing)

## परिचय

इस नोटबुक में हम सुत्र मॉडल का उपयोग OpenAI क्लाइंट के साथ विशेष रूप से हिंदी भाषा के अनुप्रयोगों के लिए कैसे करें, यह दिखाया गया है। सुत्र भारतीय भाषाओं में सामग्री को संसाधित करने और उत्पन्न करने में उत्कृष्ट है, जिससे यह हिंदी भाषा के कार्यों के लिए एक आदर्श विकल्प बन जाता है।

### आप क्या सीखेंगे

- सुत्र मॉडल के साथ काम करने के लिए OpenAI क्लाइंट को सेट करना
- हिंदी भाषा के प्रॉम्प्ट बनाना और प्रतिक्रियाओं को समझना
- विभिन्न हिंदी भाषा के उपयोग के मामलों का अन्वेषण
- हिंदी भाषा में सामग्री उत्पादन के लिए सर्वोत्तम अभ्यास

##अपने API कीज़ प्राप्त करें
शुरू करने से पहले, सुनिश्चित करें कि आपके पास:

1. एक SUTRA API की (इसे TWO AI की SUTRA API (https://www.two.ai/sutra/api) पेज पर प्राप्त करें)
2. Python और Jupyter नोटबुक्स की बुनियादी जानकारी

यह नोटबुक Google Colab में चलने के लिए डिज़ाइन की गई है, इसलिए स्थानीय Python इंस्टॉलेशन की आवश्यकता नहीं है।

## सेटअप

सबसे पहले, आवश्यक लाइब्रेरी स्थापित करें।

In [ ]:
# आवश्यक पैकेज स्थापित करें
!pip install openai requests pandas matplotlib

### आवश्यक लाइब्रेरी आयात करें


In [ ]:
# आवश्यक लाइब्रेरी आयात करें
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI
from IPython.display import display, Markdown, HTML
from google.colab import userdata

## प्रमाणीकरण

सुत्र API का उपयोग करने के लिए, आपको अपनी API कुंजी सेट करने की आवश्यकता है।

In [ ]:
# यदि आप कोड संदर्भ में इसका उपयोग करना चाहते हैं, तो उदाहरण के लिए:
os.environ["SUTRA_API_KEY"] = userdata.get('SUTRA_API_KEY')

### सुत्र के API एंडपॉइंट के साथ OpenAI क्लाइंट बनाएं


In [ ]:
# सुत्र के API एंडपॉइंट के साथ OpenAI क्लाइंट बनाएं
client = OpenAI(
    base_url="https://api.two.ai/v2",
    api_key=os.environ.get("SUTRA_API_KEY")
)

## सहायक फ़ंक्शन

हिंदी भाषा के कार्यों के लिए सुत्र API के साथ बातचीत करने के लिए सहायक फ़ंक्शन बनाएं।

In [ ]:
import time

def get_sutra_response(prompt, model="sutra-v2", temperature=0.7, max_tokens=1024):
    """
    दिए गए प्रॉम्प्ट के लिए सुत्र AI से सामान्य (non-streaming) प्रतिक्रिया प्राप्त करें।

    आर्ग्स:
        prompt (str): उपयोगकर्ता का प्रॉम्प्ट या प्रश्न।
        model (str): उपयोग करने के लिए सुत्र मॉडल।
        temperature (float): यादृच्छिकता को नियंत्रित करता है (0 से 1)।
        max_tokens (int): उत्पन्न करने के लिए अधिकतम टोकन की संख्या।

    रिटर्न्स:
        str: मॉडल की प्रतिक्रिया।
        float: प्रतिक्रिया प्राप्त करने में लगा समय।
    """
    start_time = time.time()

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )

        end_time = time.time()
        result = response.choices[0].message.content
        time_taken = end_time - start_time

        # सरल आउटपुट
        print("प्रॉम्प्ट:", prompt)
        print("प्रतिक्रिया:", result)
        print(f"समय: {time_taken:.2f} सेकंड\n")

        return result, time_taken

    except Exception as e:
        end_time = time.time()
        print("त्रुटि:", str(e))
        return f"त्रुटि: {str(e)}", end_time - start_time


def get_sutra_streaming_response(prompt, model="sutra-v2", temperature=0.7, max_tokens=1024):
    """
    दिए गए प्रॉम्प्ट के लिए सुत्र AI से स्ट्रीमिंग प्रतिक्रिया प्राप्त करें।

    आर्ग्स:
        prompt (str): उपयोगकर्ता का प्रॉम्प्ट या प्रश्न।
        model (str): उपयोग करने के लिए सुत्र मॉडल।
        temperature (float): यादृच्छिकता को नियंत्रित करता है (0 से 1)।
        max_tokens (int): उत्पन्न करने के लिए अधिकतम टोकन की संख्या।

    रिटर्न्स:
        str: पूरी मॉडल प्रतिक्रिया।
        float: प्रतिक्रिया प्राप्त करने में लगा कुल समय।
    """
    start_time = time.time()

    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            stream=True
        )

        print("प्रॉम्प्ट:", prompt)
        print("प्रतिक्रिया:")

        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                full_response += content
                print(content, end="")

        print("\n")
        time_taken = time.time() - start_time
        print(f"समय: {time_taken:.2f} सेकंड\n")

        return full_response, time_taken

    except Exception as e:
        end_time = time.time()
        print("त्रुटि:", str(e))
        return f"त्रुटि: {str(e)}", end_time - start_time


## 1. बुनियादी हिंदी भाषा उत्पादन

आइए सुत्र के साथ हिंदी पाठ उत्पन्न करने के कुछ बुनियादी उदाहरणों से शुरू करें।

In [ ]:
# सरल हिंदी पाठ उत्पादन
hindi_prompt = "नमस्ते, मेरा नाम राहुल है। आप कैसे हैं?"

response, time_taken = get_sutra_response(hindi_prompt)

प्रॉम्प्ट: नमस्ते, मेरा नाम राहुल है। आप कैसे हैं?
प्रतिक्रिया: नमस्ते राहुल! मैं ठीक हूँ, धन्यवाद। आप कैसे हैं?
समय: 2.53 सेकंड



## 2. हिंदी में जानकारी पुनर्प्राप्ति

आइए हिंदी में जानकारी पुनर्प्राप्त करने के लिए सुत्र का उपयोग करें।

In [ ]:
# हिंदी में जानकारी पुनर्प्राप्ति
info_prompt = "भारत के पांच प्रमुख त्योहारों के बारे में बताएं और उनके महत्व पर प्रकाश डालें।"

response, time_taken = get_sutra_response(info_prompt)

प्रॉम्प्ट: भारत के पांच प्रमुख त्योहारों के बारे में बताएं और उनके महत्व पर प्रकाश डालें।
प्रतिक्रिया: भारत में अनेक त्योहार मनाए जाते हैं, जो देश की विविधता और सांस्कृतिक धरोहर को दर्शाते हैं। यहाँ पांच प्रमुख त्योहारों और उनके महत्व के बारे में जानकारी दी गई है:

1. **दीवाली (दीपावली)**:
   - **महत्व**: दीवाली, जिसे दीपावली भी कहा जाता है, हिंदू धर्म का एक प्रमुख त्यौहार है। यह विशेष रूप से भगवान राम के अयोध्या लौटने और रावण पर विजय प्राप्त करने के उपलक्ष्य में मनाया जाता है। यह अंधकार से प्रकाश की ओर जाने का प्रतीक है।
   - **विशेषताएँ**: घरों की सफाई, दीप जलाना, पटाखे फोड़ना और मिठाई बाँटना इस त्योहार की विशेषताएँ हैं।

2. **होली**:
   - **महत्व**: होली, रंगों का त्योहार, बुराई पर अच्छाई की विजय का प्रतीक है। इसे फाल्गुन मास की पूर्णिमा को मनाया जाता है और यह वसंत ऋतु के आगमन का स्वागत करता है।
   - **विशेषताएँ**: लोग एक-दूसरे पर रंग डालते हैं, गाते-बजाते हैं और मिठाइयाँ बांटते हैं। यह प्रेम और भाईचारे का संदेश फैलाता है।

3. **ईद-उल-फितर**:
   - **महत्व**: ईद-उल-फितर रमजान के महीन

## 3. हिंदी रचनात्मक लेखन

सुत्र हिंदी में रचनात्मक सामग्री उत्पन्न कर सकता है।

In [ ]:
# हिंदी में रचनात्मक लेखन
creative_prompt = "हिमालय पर्वत पर एक छोटी कविता लिखें।"

response, time_taken = get_sutra_response(creative_prompt, temperature=0.8)

प्रॉम्प्ट: हिमालय पर्वत पर एक छोटी कविता लिखें।
प्रतिक्रिया: हिमालय की ऊँचाइयों में, बसा है एक संसार,  
बर्फीली चादर ओढ़े, जैसे कोई अनमोल बहार।  
गगनचुंबी चोटी पर, बादल करते हैं नृत्य,  
सूरज की किरणें छूकर, बिखेरती हैं ज्योत्स्ना के रंग।

नदियाँ बहतीं यहाँ, जैसे गीत गाती हों,  
हरियाली के हथेलियों पर, जीवन फूल खिलाती हों।  
तपस्वियों की साधना, गूंजती है इन घाटियों में,  
हिमालय की गोद में, छिपा है सुख का सागर यहाँ।
समय: 3.18 सेकंड



## 4. हिंदी सारांशीकरण

सुत्र हिंदी पाठ को प्रभावी ढंग से सारांशित कर सकता है।

In [ ]:
# सारांशित करने के लिए पाठ
long_text = """
भारत की स्वतंत्रता का इतिहास एक लंबा और जटिल है। भारत में ब्रिटिश शासन की शुरुआत 1757 में प्लासी के युद्ध के बाद हुई, जब ईस्ट इंडिया कंपनी ने बंगाल के नवाब सिराज-उद-दौला को हराया। इसके बाद, कंपनी ने धीरे-धीरे अपने प्रभाव क्षेत्र का विस्तार किया और 1858 तक, भारत सीधे ब्रिटिश क्राउन के अधीन आ गया।

1857 का विद्रोह, जिसे प्रथम स्वतंत्रता संग्राम के रूप में भी जाना जाता है, ब्रिटिश शासन के खिलाफ पहला बड़ा विद्रोह था। हालांकि यह विद्रोह असफल रहा, लेकिन इसने भारतीयों में राष्ट्रीय चेतना को जगाया।

1885 में भारतीय राष्ट्रीय कांग्रेस की स्थापना के साथ, स्वतंत्रता आंदोलन ने एक संगठित रूप ले लिया। महात्मा गांधी के नेतृत्व में, अहिंसात्मक प्रतिरोध और सत्याग्रह के माध्यम से स्वतंत्रता की मांग की गई।

1942 में 'भारत छोड़ो आंदोलन' शुरू हुआ, जिसने ब्रिटिश शासन को भारत छोड़ने के लिए मजबूर किया। द्वितीय विश्व युद्ध के बाद, ब्रिटेन की आर्थिक स्थिति कमजोर हो गई और अंततः 15 अगस्त 1947 को भारत को स्वतंत्रता मिली, हालांकि देश का विभाजन भारत और पाकिस्तान में हो गया।
"""

summarize_prompt = f"निम्नलिखित पाठ का सारांश करें (100 शब्दों से कम में):\n\n{long_text}"

summary, time_taken = get_sutra_response(summarize_prompt, temperature=0.3)

प्रॉम्प्ट: निम्नलिखित पाठ का सारांश करें (100 शब्दों से कम में):


भारत की स्वतंत्रता का इतिहास एक लंबा और जटिल है। भारत में ब्रिटिश शासन की शुरुआत 1757 में प्लासी के युद्ध के बाद हुई, जब ईस्ट इंडिया कंपनी ने बंगाल के नवाब सिराज-उद-दौला को हराया। इसके बाद, कंपनी ने धीरे-धीरे अपने प्रभाव क्षेत्र का विस्तार किया और 1858 तक, भारत सीधे ब्रिटिश क्राउन के अधीन आ गया।

1857 का विद्रोह, जिसे प्रथम स्वतंत्रता संग्राम के रूप में भी जाना जाता है, ब्रिटिश शासन के खिलाफ पहला बड़ा विद्रोह था। हालांकि यह विद्रोह असफल रहा, लेकिन इसने भारतीयों में राष्ट्रीय चेतना को जगाया।

1885 में भारतीय राष्ट्रीय कांग्रेस की स्थापना के साथ, स्वतंत्रता आंदोलन ने एक संगठित रूप ले लिया। महात्मा गांधी के नेतृत्व में, अहिंसात्मक प्रतिरोध और सत्याग्रह के माध्यम से स्वतंत्रता की मांग की गई।

1942 में 'भारत छोड़ो आंदोलन' शुरू हुआ, जिसने ब्रिटिश शासन को भारत छोड़ने के लिए मजबूर किया। द्वितीय विश्व युद्ध के बाद, ब्रिटेन की आर्थिक स्थिति कमजोर हो गई और अंततः 15 अगस्त 1947 को भारत को स्वतंत्रता मिली, हालांकि देश का विभाजन भारत 

## 5. हिंदी प्रश्न उत्तर

आइए हिंदी में प्रश्न उत्तर के लिए सुत्र का उपयोग करें।

In [ ]:
# प्रश्न उत्तर के लिए संदर्भ
context = """
भारतीय संविधान दुनिया का सबसे लंबा लिखित संविधान है। इसे 26 नवंबर 1949 को अपनाया गया था और 26 जनवरी 1950 को लागू किया गया था। डॉ. भीमराव अंबेडकर को भारतीय संविधान का मुख्य वास्तुकार माना जाता है। संविधान में मूल रूप से 395 अनुच्छेद, 22 भाग और 8 अनुसूचियां थीं। वर्तमान में, इसमें 448 अनुच्छेद, 25 भाग और 12 अनुसूचियां हैं। संविधान की प्रस्तावना में 'समाजवादी', 'धर्मनिरपेक्ष' और 'अखंडता' शब्द 42वें संशोधन (1976) द्वारा जोड़े गए थे।
"""

qa_prompt = f"निम्नलिखित जानकारी के आधार पर प्रश्नों के उत्तर दें:\n\n{context}\n\nप्रश्न 1: भारतीय संविधान कब लागू हुआ?\nप्रश्न 2: भारतीय संविधान का मुख्य वास्तुकार कौन था?\nप्रश्न 3: 42वें संशोधन द्वारा प्रस्तावना में कौन से शब्द जोड़े गए थे?"

answers, time_taken = get_sutra_response(qa_prompt, temperature=0.2)

प्रॉम्प्ट: निम्नलिखित जानकारी के आधार पर प्रश्नों के उत्तर दें:


भारतीय संविधान दुनिया का सबसे लंबा लिखित संविधान है। इसे 26 नवंबर 1949 को अपनाया गया था और 26 जनवरी 1950 को लागू किया गया था। डॉ. भीमराव अंबेडकर को भारतीय संविधान का मुख्य वास्तुकार माना जाता है। संविधान में मूल रूप से 395 अनुच्छेद, 22 भाग और 8 अनुसूचियां थीं। वर्तमान में, इसमें 448 अनुच्छेद, 25 भाग और 12 अनुसूचियां हैं। संविधान की प्रस्तावना में 'समाजवादी', 'धर्मनिरपेक्ष' और 'अखंडता' शब्द 42वें संशोधन (1976) द्वारा जोड़े गए थे।


प्रश्न 1: भारतीय संविधान कब लागू हुआ?
प्रश्न 2: भारतीय संविधान का मुख्य वास्तुकार कौन था?
प्रश्न 3: 42वें संशोधन द्वारा प्रस्तावना में कौन से शब्द जोड़े गए थे?
प्रतिक्रिया: प्रश्न 1: भारतीय संविधान 26 जनवरी 1950 को लागू हुआ था।

प्रश्न 2: भारतीय संविधान का मुख्य वास्तुकार डॉ. भीमराव अंबेडकर थे।

प्रश्न 3: 42वें संशोधन (1976) द्वारा प्रस्तावना में 'समाजवादी', 'धर्मनिरपेक्ष' और 'अखंडता' शब्द जोड़े गए थे।
समय: 3.30 सेकंड



## 6. हिंदी-अंग्रेजी अनुवाद

सुत्र हिंदी और अंग्रेजी के बीच प्रभावी ढंग से अनुवाद कर सकता है।

In [ ]:
# हिंदी से अंग्रेजी अनुवाद
hindi_text = "भारत एक विविधतापूर्ण देश है जहां अनेक भाषाएँ, धर्म और संस्कृतियाँ एक साथ फलती-फूलती हैं। यहाँ की समृद्ध परंपराएँ और इतिहास इसे विश्व में एक अनोखा स्थान देते हैं।"

translate_hi_to_en_prompt = f"निम्नलिखित हिंदी पाठ का अंग्रेजी में अनुवाद करें:\n\n{hindi_text}"

translation_en, time_taken = get_sutra_response(translate_hi_to_en_prompt)

प्रॉम्प्ट: निम्नलिखित हिंदी पाठ का अंग्रेजी में अनुवाद करें:

भारत एक विविधतापूर्ण देश है जहां अनेक भाषाएँ, धर्म और संस्कृतियाँ एक साथ फलती-फूलती हैं। यहाँ की समृद्ध परंपराएँ और इतिहास इसे विश्व में एक अनोखा स्थान देते हैं।
प्रतिक्रिया: India is a diverse country where numerous languages, religions, and cultures thrive together. Its rich traditions and history give it a unique place in the world.
समय: 2.04 सेकंड



In [ ]:
# अंग्रेजी से हिंदी अनुवाद
english_text = "Artificial intelligence is transforming the way we live and work. It has applications in healthcare, education, transportation, and many other fields. As technology advances, it's important to ensure that AI is developed ethically and responsibly."

translate_en_to_hi_prompt = f"निम्नलिखित अंग्रेजी पाठ का हिंदी में अनुवाद करें:\n\n{english_text}"

translation_hi, time_taken = get_sutra_response(translate_en_to_hi_prompt)

प्रॉम्प्ट: निम्नलिखित अंग्रेजी पाठ का हिंदी में अनुवाद करें:

Artificial intelligence is transforming the way we live and work. It has applications in healthcare, education, transportation, and many other fields. As technology advances, it's important to ensure that AI is developed ethically and responsibly.
प्रतिक्रिया: कृत्रिम बुद्धिमत्ता हमारे जीने और काम करने के तरीके को बदल रही है। इसके स्वास्थ्य देखभाल, शिक्षा, परिवहन और कई अन्य क्षेत्रों में अनुप्रयोग हैं। जैसे-जैसे तकनीक आगे बढ़ती है, यह सुनिश्चित करना महत्वपूर्ण है कि एआई का विकास नैतिक और जिम्मेदार तरीके से किया जाए।
समय: 2.58 सेकंड



## 7. हिंदी भावात्मक विश्लेषण

आइए हिंदी पाठ के भावात्मक विश्लेषण के लिए सुत्र का उपयोग करें।

In [ ]:
# भावात्मक विश्लेषण के लिए हिंदी समीक्षाएँ
reviews = [
    "यह फिल्म बहुत अच्छी थी। कहानी रोचक थी और अभिनय शानदार था।",
    "खाना बिल्कुल स्वादिष्ट नहीं था और सेवा भी बहुत खराब थी।",
    "उत्पाद ठीक-ठाक है, लेकिन कीमत थोड़ी ज्यादा है।",
    "मुझे यह किताब पसंद आई, लेखक ने विषय को बहुत अच्छे से समझाया है।",
    "यात्रा निराशाजनक थी, होटल गंदा था और स्टाफ अमित्रवत था।"
]

sentiment_prompt = f"निम्नलिखित हिंदी समीक्षाओं का भावात्मक विश्लेषण करें और प्रत्येक को 'सकारात्मक', 'नकारात्मक', या 'तटस्थ' के रूप में वर्गीकृत करें:\n\n"

for i, review in enumerate(reviews):
    sentiment_prompt += f"{i+1}. {review}\n"

sentiment_analysis, time_taken = get_sutra_response(sentiment_prompt, temperature=0.1)

प्रॉम्प्ट: निम्नलिखित हिंदी समीक्षाओं का भावात्मक विश्लेषण करें और प्रत्येक को 'सकारात्मक', 'नकारात्मक', या 'तटस्थ' के रूप में वर्गीकृत करें:

1. यह फिल्म बहुत अच्छी थी। कहानी रोचक थी और अभिनय शानदार था।
2. खाना बिल्कुल स्वादिष्ट नहीं था और सेवा भी बहुत खराब थी।
3. उत्पाद ठीक-ठाक है, लेकिन कीमत थोड़ी ज्यादा है।
4. मुझे यह किताब पसंद आई, लेखक ने विषय को बहुत अच्छे से समझाया है।
5. यात्रा निराशाजनक थी, होटल गंदा था और स्टाफ अमित्रवत था।

प्रतिक्रिया: 1. **सकारात्मक** - यह फिल्म बहुत अच्छी थी। कहानी रोचक थी और अभिनय शानदार था।
2. **नकारात्मक** - खाना बिल्कुल स्वादिष्ट नहीं था और सेवा भी बहुत खराब थी।
3. **तटस्थ** - उत्पाद ठीक-ठाक है, लेकिन कीमत थोड़ी ज्यादा है।
4. **सकारात्मक** - मुझे यह किताब पसंद आई, लेखक ने विषय को बहुत अच्छे से समझाया है।
5. **नकारात्मक** - यात्रा निराशाजनक थी, होटल गंदा था और स्टाफ अमित्रवत था।
समय: 3.46 सेकंड



## 8. विभिन्न प्रारूपों के लिए हिंदी सामग्री उत्पादन

सुत्र विभिन्न प्रारूपों में हिंदी सामग्री उत्पन्न कर सकता है।

In [ ]:
# हिंदी ब्लॉग पोस्ट उत्पन्न करें
blog_prompt = "'भारतीय व्यंजन और उनका स्वास्थ्य पर प्रभाव' विषय पर एक ब्लॉग पोस्ट लिखें। इसमें शीर्षक, परिचय, मुख्य बिंदु और निष्कर्ष शामिल करें।"

blog_post = get_sutra_streaming_response(blog_prompt, temperature=0.7)

प्रॉम्प्ट: 'भारतीय व्यंजन और उनका स्वास्थ्य पर प्रभाव' विषय पर एक ब्लॉग पोस्ट लिखें। इसमें शीर्षक, परिचय, मुख्य बिंदु और निष्कर्ष शामिल करें।
प्रतिक्रिया:
### भारतीय व्यंजन और उनका स्वास्थ्य पर प्रभाव

#### परिचय
भारतीय व्यंजन अपनी विविधता, रंग-बिरंगे स्वाद और सुगंध के लिए प्रसिद्ध हैं। भारत की संस्कृति और परंपराओं का एक अभिन्न हिस्सा होने के नाते, ये व्यंजन न केवल स्वादिष्ट होते हैं, बल्कि इनमें पोषण भी भरपूर होता है। इस ब्लॉग पोस्ट में हम भारतीय व्यंजनों के विभिन्न प्रकारों और उनके स्वास्थ्य पर प्रभाव का विश्लेषण करेंगे।

#### मुख्य बिंदु

1. **विविधता और क्षेत्रीय विशेषताएँ**  
   भारत के विभिन्न राज्यों में व्यंजनों की विविधता अद्भुत है। उत्तर भारत में रोटी, दाल और सब्जियाँ लोकप्रिय हैं, जबकि दक्षिण भारत में चावल, सांबर और डोसा मुख्य भोजन हैं। हर क्षेत्र के व्यंजनों में स्थानीय सामग्री और मसालों का प्रयोग किया जाता है, जो उन्हें विशेष बनाता है।

2. **मसालों का महत्व**  
   भारतीय व्यंजनों में मसाले न केवल स्वाद बढ़ाते हैं, बल्कि स्वास्थ्य के लिए भी फायदेमंद होते हैं। जैसे हल्दी, जो

## 9. निष्कर्ष

इस नोटबुक में, हमने हिंदी भाषा के अनुप्रयोगों के लिए OpenAI क्लाइंट के साथ सुत्र मॉडल का उपयोग करने का अन्वेषण किया है। हमने विभिन्न उपयोग के मामलों को कवर किया है, जिनमें शामिल हैं:

- बुनियादी हिंदी पाठ उत्पादन
- हिंदी में जानकारी पुनर्प्राप्ति
- हिंदी रचनात्मक लेखन
- हिंदी पाठ सारांशीकरण
- हिंदी प्रश्न उत्तर
- हिंदी-अंग्रेजी अनुवाद
- हिंदी भावात्मक विश्लेषण

हिंदी और अन्य भारतीय भाषाओं में सुत्र की मजबूत क्षमताएं इसे हिंदी-भाषी उपयोगकर्ताओं को लक्षित करने वाले या हिंदी भाषा प्रसंस्करण की आवश्यकता वाले अनुप्रयोगों के लिए एक उत्कृष्ट विकल्प बनाती हैं।

## 10. अतिरिक्त संसाधन

हिंदी भाषा के साथ सुत्र का उपयोग करने के बारे में अधिक जानने के लिए, निम्नलिखित संसाधनों का उपयोग करें:

1. [सुत्र API दस्तावेज़ीकरण](https://docs.two.ai)
2. [सुत्र मॉडल वेरिएंट](https://docs.two.ai/models)
3. [बहुभाषी क्षमताओं के बारे में अधिक जानकारी](https://docs.two.ai/multilingual)

हिंदी और अन्य भारतीय भाषाओं के लिए सुत्र का उपयोग करके अपने स्वयं के अनुप्रयोगों का निर्माण करने के लिए प्रोत्साहित किया जाता है!